In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from matplotlib import collections  as mc
import numpy as np

import geopandas as gpd
from mpl_toolkits.axes_grid1 import make_axes_locatable
from matplotlib import cm
from matplotlib.colors import Normalize 

from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.linear_model import LinearRegression
from sklearn import metrics
from sklearn.preprocessing import StandardScaler

from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier, export_graphviz
#import graphviz
from IPython.display import SVG

import warnings
import string

warnings.filterwarnings('ignore')

%matplotlib inline

In [2]:
sd = pd.read_csv('/Users/staufferkm/Desktop/gitrepos/fasttt/SanDiegoPoliceRipaStopsMerged.csv')

In [27]:
sd.head()

,stop_id,pid,idid,race_category,racecode,search_conducted,contraband_found,total_beat_population,black_beat_population,latinx_beat_population,...,type of property seized,basis for property seizure,baton used?,taser used?,impact projectile used?,physical or vehicle contact?,firearm pointed?,firearm discharged?,chemical spray used?,canine contact?
0,124066,1,124066x1,Latinx,2,1,0,14176.0,210.0,10691.0,...,Suspected Stolen property,Impound of vehicle,0,0,Impact projectile discharged or used,0,Firearm pointed at person,0,0,0
1,60843,1,60843x1,White,1,1,0,12893.0,691.0,3180.0,...,0,0,0,0,Impact projectile discharged or used,Physical or Vehicle contact,0,0,0,0
2,73092,1,73092x1,White,1,1,0,11512.0,1639.0,6946.0,...,0,0,0,Electronic control device used,Impact projectile discharged or used,Physical or Vehicle contact,Firearm pointed at person,0,0,0
3,16137,1,16137x1,White,1,1,1,12893.0,691.0,3180.0,...,Drug Paraphernalia,Contraband,0,0,Impact projectile discharged or used,Physical or Vehicle contact,Firearm pointed at person,0,0,0
4,64190,1,64190x1,Latinx,2,1,1,26700.0,203.0,25088.0,...,0,0,0,Electronic control device used,Impact projectile discharged or used,Physical or Vehicle contact,0,0,0,0


## Keep only columns important for now
also rename columns:  race to driver_race, and beat to location_variable

In [3]:
sd_short = sd[['race', 'search_conducted', 'contraband_found', 'beat', 'Reason for Stop']]
sd_short.head()

,race,search_conducted,contraband_found,beat,Reason for Stop
0,Hispanic/Latino/a,1,0,722,Reasonable Suspicion
1,White,1,0,624,Reasonable Suspicion
2,White,1,0,826,Reasonable Suspicion
3,White,1,1,624,Reasonable Suspicion
4,Hispanic/Latino/a,1,1,712,Knowledge of outstanding arrest warrant/wanted...


In [4]:
sd_short.columns = ['driver_race', 'search_conducted', 'contraband_found', 'location_variable', 'Reason_for_Stop']
sd_short.head()

,driver_race,search_conducted,contraband_found,location_variable,Reason_for_Stop
0,Hispanic/Latino/a,1,0,722,Reasonable Suspicion
1,White,1,0,624,Reasonable Suspicion
2,White,1,0,826,Reasonable Suspicion
3,White,1,1,624,Reasonable Suspicion
4,Hispanic/Latino/a,1,1,712,Knowledge of outstanding arrest warrant/wanted...


In [5]:
sd_short.driver_race.unique()

array(['Hispanic/Latino/a', 'White', 'Black/African American', 'Asian',
       'Middle Eastern or South Asian', 'Pacific Islander',
       'Native American'], dtype=object)

# Rename Hispanic/Latino/a to Hispanic and Black/African American to Black

In [6]:
race_dict = dict([('Hispanic/Latino/a', 'Hispanic'), 
                    ('Black/African American','Black') ])

In [7]:
sd_short = sd_short.replace({"driver_race": race_dict})
sd_short.head()

,driver_race,search_conducted,contraband_found,location_variable,Reason_for_Stop
0,Hispanic,1,0,722,Reasonable Suspicion
1,White,1,0,624,Reasonable Suspicion
2,White,1,0,826,Reasonable Suspicion
3,White,1,1,624,Reasonable Suspicion
4,Hispanic,1,1,712,Knowledge of outstanding arrest warrant/wanted...


## Now I want to only look at the traffic stops

In [8]:
sd_traffic = sd_short.query('Reason_for_Stop == "Traffic Violation"')
sd_traffic.location_variable.unique()

array([824, 831, 832, 834, 837, 614, 521, 441, 628, 451, 444, 611, 313,
       713, 123, 122, 723, 322, 712, 512, 622, 838, 836, 813, 324, 121,
       826, 833, 436, 315, 515, 242, 725, 511, 621, 999, 722, 931, 827,
       446, 442, 623, 434, 821, 321, 432, 431, 311, 111, 618, 437, 116,
       724, 517, 447, 114, 115, 613, 326, 112, 531, 627, 452, 443, 113,
       721, 514, 516, 822, 528, 835, 445, 541, 615, 523, 312, 711, 624,
       314, 524, 323, 327, 438, 625, 235, 439, 811, 526, 934, 839, 626,
       124, 823, 522, 433, 529, 841, 932, 243, 935, 933, 527, 233, 825,
       231, 241, 828, 246, 937, 232, 525, 234, 814, 714, 126, 617, 518,
       325, 612, 316, 936, 435, 812, 245, 125, 829])

# We need to create a new table with the following columns:
    driver_race	location_variable	num_stops	num_searches	num_hits	state	city	county

In [9]:
stopnum = []
stopnum = sd_traffic.groupby(['location_variable', 'driver_race']).count().reset_index()
stopnum1 = stopnum[['location_variable', 'driver_race', 'Reason_for_Stop']]
stopnum1.columns = ['location_variable', 'driver_race', 'num_stops']
stopnum1.head()

,location_variable,driver_race,num_stops
0,111,Asian,106
1,111,Black,130
2,111,Hispanic,339
3,111,Middle Eastern or South Asian,66
4,111,Native American,2


In [10]:
num_searches = []
num_searches = sd_traffic.groupby(['location_variable', 'driver_race', 'search_conducted']).count().reset_index()
num_searches_pos = num_searches.query('search_conducted == "1"')
num_searches_1 = num_searches_pos[['location_variable', 'driver_race', 'Reason_for_Stop']]
num_searches_1.columns = ['location_variable', 'driver_race', 'num_searches']
num_searches_1.head()

,location_variable,driver_race,num_searches
1,111,Asian,5
3,111,Black,14
5,111,Hispanic,25
7,111,Middle Eastern or South Asian,1
9,111,Native American,1


In [11]:
num_hits = []
num_hits = sd_traffic.groupby(['location_variable', 'driver_race', 'contraband_found']).count().reset_index()
num_hits_pos = num_hits.query('contraband_found == "1"')
num_hits_1 = num_hits_pos[['location_variable', 'driver_race', 'Reason_for_Stop']]
num_hits_1.columns = ['location_variable', 'driver_race', 'num_hits']
num_hits_1.head()

,location_variable,driver_race,num_hits
1,111,Asian,2
3,111,Black,2
5,111,Hispanic,7
8,111,Native American,1
11,111,White,16


## Join the data frame on the columns of location_variable and driver_race

In [12]:
result1 = pd.merge(stopnum1, num_searches_1, how='outer', on=['location_variable', 'driver_race'])
result1["num_searches"].fillna("0", inplace = True)
result1.head(20)

,location_variable,driver_race,num_stops,num_searches
0,111,Asian,106,5
1,111,Black,130,14
2,111,Hispanic,339,25
3,111,Middle Eastern or South Asian,66,1
4,111,Native American,2,1
5,111,Pacific Islander,11,0
6,111,White,578,54
7,112,Asian,32,0
8,112,Black,39,3
9,112,Hispanic,102,5


In [13]:
result = pd.merge(result1, num_hits_1, how='outer', on=['location_variable', 'driver_race'])
result["num_hits"].fillna("0", inplace = True)
result.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 774 entries, 0 to 773
Data columns (total 5 columns):
location_variable    774 non-null int64
driver_race          774 non-null object
num_stops            774 non-null int64
num_searches         774 non-null object
num_hits             774 non-null object
dtypes: int64(2), object(3)
memory usage: 36.3+ KB


In [96]:
result.to_csv('/Users/staufferkm/Desktop/gitrepos/fasttt/SanDiegoPoliceRipaStopsTrafficOnly.csv', sep='\t', index=False)


In [14]:
result_slim = result.query(('driver_race == "Black" or driver_race == "White" or driver_race == "Hispanic"'))
result_slim.driver_race.unique()

array(['Black', 'Hispanic', 'White'], dtype=object)

In [30]:
result_slim.to_csv('/Users/staufferkm/Desktop/gitrepos/fasttt/SanDiegoPoliceRipaStopsTrafficOnly_3Race.csv', sep='\t', index=False)


I want to see if there are any rows where the num_hits is larger than the num_searches

In [16]:
result_slim.head()

,location_variable,driver_race,num_stops,num_searches,num_hits
1,111,Black,130,14,2
2,111,Hispanic,339,25,7
6,111,White,578,54,16
8,112,Black,39,3,1
9,112,Hispanic,102,5,0


In [26]:
takeout = []
for row in result_slim.iterrows():
    vals = row[1]
    search = int(vals['num_searches'])
    hit = int(vals['num_hits'])
    if search < hit:
        takeout.append(vals[])
takeout

[location_variable      325
 driver_race          Black
 num_stops               10
 num_searches             0
 num_hits                 1
 Name: 195, dtype: object, location_variable      615
 driver_race          Black
 num_stops               17
 num_searches             0
 num_hits                 1
 Name: 466, dtype: object, location_variable         823
 driver_race          Hispanic
 num_stops                  80
 num_searches               11
 num_hits                   15
 Name: 623, dtype: object, location_variable      823
 driver_race          White
 num_stops               30
 num_searches             5
 num_hits                 9
 Name: 626, dtype: object, location_variable      826
 driver_race          Black
 num_stops              397
 num_searches            66
 num_hits                80
 Name: 640, dtype: object, location_variable         826
 driver_race          Hispanic
 num_stops                 396
 num_searches               64
 num_hits                   71


I need to create the stops per precinct file now

In [30]:
three = pd.read_csv('/Users/staufferkm/Desktop/gitrepos/fasttt/SanDiegoPoliceRipaStopsTrafficOnly_3Race.txt', sep="\t")

In [31]:
three.head()

,driver_race,location_variable,num_stops,num_searches,num_hits,state,city,county
0,Black,111,130,14,2,CA,San_Diego,San_Diego
1,Black,112,39,3,1,CA,San_Diego,San_Diego
2,Black,113,26,3,0,CA,San_Diego,San_Diego
3,Black,114,100,9,3,CA,San_Diego,San_Diego
4,Black,115,75,3,1,CA,San_Diego,San_Diego


In [32]:
bybeat = three.groupby(['location_variable']).sum().reset_index()
bybeat.head()

,location_variable,num_stops,num_searches,num_hits
0,111,1047,93,25
1,112,430,33,9
2,113,350,29,10
3,114,949,78,21
4,115,823,30,11


In [33]:
bybeat1 = bybeat[['location_variable', 'num_stops']]
bybeat1.head()

,location_variable,num_stops
0,111,1047
1,112,430
2,113,350
3,114,949
4,115,823


In [34]:
bybeat1.columns = ['location_variable', 'total_stops']
bybeat1.head()

,location_variable,total_stops
0,111,1047
1,112,430
2,113,350
3,114,949
4,115,823


In [35]:
bybeat1.to_csv('/Users/staufferkm/Desktop/gitrepos/fasttt/stops_per_precinctRipaStopsTrafficOnly.csv', sep=',', index=False)